In [ ]:
!python3 -m pip install git+https://github.com/zebrabug/tensortrade.git

In [ ]:
!git clone https://github.com/zebrabug/tensortrade.git

In [3]:
!cd tensortrade/examples

In [ ]:
!pip install -r tensortrade/requirements.txt

In [5]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [6]:
!pip install ta

  Created wheel for ta: filename=ta-0.7.0-cp37-none-any.whl size=28716 sha256=a8eff8aded473e2d83cc2b3a90d5eae41f9a299701d999f5254c791f85bd3723
  Stored in directory: /root/.cache/pip/wheels/dd/88/30/de9553fb54a474eb7480b937cdbb140bdda613d29cf4da7994
Successfully built ta


In [7]:
!pip install tensorforce

     |████████████████████████████████| 307kB 4.0MB/s 
     |████████████████████████████████| 10.3MB 18.0MB/s 
     |████████████████████████████████| 2.2MB 41.8MB/s 
     |████████████████████████████████| 1.6MB 49.6MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
  Created wheel for gym: filename=gym-0.18.3-cp37-none-any.whl size=1657514 sha256=aeb886b59fcc32acd371aa56241dae2a53c5149fed35dbe1ca1125e6717ef646
  Stored in directory: /root/.cache/pip/wheels/93/c2/4c/2b4c9b85119994837c08315c9415d71008325b7004d385b418
Successfully built gym
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3:
   

In [10]:
import ta

import pandas as pd

from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.exchanges import Exchange, ExchangeOptions
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC, RUR
from tensortrade.oms.wallets import Wallet, Portfolio
import tensortrade.env.default as default
from tensortrade.agents import DQNAgent
import matplotlib.pyplot as plt
from finrl.model.models import DRLAgent
from  IPython.core.debugger import set_trace

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

df = pd.read_pickle('./tensortrade/examples/data/OHLC_deals_df.pkl')
df = df.loc[df['Close'].notnull(), :]
df.rename(columns = {
    'Time':'date',
    'Open':'open',
    'Close':'close',
    'Low':'low',
    'High':'high',
    'Volume':'volume'
}, inplace = True)

df['h'] = df.date.dt.hour
df['d'] = df.date.dt.date


In [19]:
import numpy as np
N = 30
rand_dates = np.random.choice(df.date.dt.date.unique(), N)
rand_hours = np.random.choice([10,12,13,14,15,16], N)

In [20]:
agent = None
l = []
a = 73000

for i in range(N):
    
    #set_trace()
    tmp = df.loc[(df.d == rand_dates[i]) & (df.h >= rand_hours[i]),:]
    if tmp.shape[0] < 360:
        continue

    dataset = ta.add_all_ta_features(tmp, 'open', 'high', 'low', 'close', 'volume', fillna=True)
    dataset.drop(columns = ['h', 'd'], inplace = True)
    price_history = dataset[['date', 'open', 'high', 'low', 'close', 'volume']]  # chart data
    dataset.drop(columns=['date', 'open', 'high', 'low', 'close', 'volume'], inplace=True)

    micex = Exchange("MICEX", 
                    service=execute_order, 
                    options=ExchangeOptions(commission = 0.003, #0.003,
                                            min_trade_size = 1e-6,
                                            max_trade_size = 1e6,
                                            min_trade_price = 1e-8,
                                            max_trade_price= 1e8,
                                            is_live=False)  )(
                                                Stream.source(price_history['close'].tolist(), dtype="float").rename("RUR-USD"))

    portfolio = Portfolio(RUR, [
        Wallet(micex, 0 * USD),
        Wallet(micex, a * RUR),
    ])

    with NameSpace("MICEX"):
        streams = [Stream.source(dataset[c].tolist(), dtype="float").rename(c) for c in dataset.columns]

    feed = DataFeed(streams)
    feed.next()

    env = default.create(
        portfolio=portfolio,
        action_scheme="simple",
        reward_scheme="simple",
        feed=feed,
        renderer="screen-log",  # ScreenLogger used with default settings
        window_size=20
    )

    if agent is None:
        agent = DQNAgent(env)
    else:
        agent = DQNAgent(env,policy_network=agent.policy_network)
    agent.train(n_episodes=1, n_steps=360, render_interval=10)

    
    perf = pd.DataFrame(portfolio.performance).transpose()[['net_worth']]
    #perf.net_worth.plot()
    plt.show()
    l += [perf['net_worth'][perf.shape[0]-1] - perf['net_worth'][0]]

print('Рещультаты:')
print(f'Средняя прибыльность сессии {np.mean(l)}, стандартное отклонение {np.std(l)}, количество сессий {len(l)}')

====      AGENT ID: 2707c72f-04e4-48cb-8501-7e1141c1144b      ====
[2021-05-23 9:24:10 AM] Episode: 1/1 Step: 131/360
[2021-05-23 9:24:11 AM] Episode: 1/1 Step: 141/360
[2021-05-23 9:24:12 AM] Episode: 1/1 Step: 151/360
[2021-05-23 9:24:12 AM] Episode: 1/1 Step: 161/360
[2021-05-23 9:24:13 AM] Episode: 1/1 Step: 171/360
[2021-05-23 9:24:14 AM] Episode: 1/1 Step: 181/360
[2021-05-23 9:24:15 AM] Episode: 1/1 Step: 191/360
[2021-05-23 9:24:16 AM] Episode: 1/1 Step: 201/360
[2021-05-23 9:24:16 AM] Episode: 1/1 Step: 211/360
[2021-05-23 9:24:17 AM] Episode: 1/1 Step: 221/360
[2021-05-23 9:24:18 AM] Episode: 1/1 Step: 231/360
[2021-05-23 9:24:19 AM] Episode: 1/1 Step: 241/360
[2021-05-23 9:24:19 AM] Episode: 1/1 Step: 251/360
[2021-05-23 9:24:20 AM] Episode: 1/1 Step: 261/360
[2021-05-23 9:24:21 AM] Episode: 1/1 Step: 271/360
[2021-05-23 9:24:22 AM] Episode: 1/1 Step: 281/360
[2021-05-23 9:24:23 AM] Episode: 1/1 Step: 291/360
[2021-05-23 9:24:23 AM] Episode: 1/1 Step: 301/360
[2021-05-23 9:2